In [1]:
#encoding=utf-8
import jieba
import collections
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from math import log10

jieba.set_dictionary('./dict.txt')

In [22]:
def preprocessing(articles):
    dataset = list()
    for article in articles[:10000]: # 先少一點
        sentences = article[:-1].split('\t') # sentences: list
        for i in range(len(sentences)): # iterate 時不會直接改到原本容器, 所以要用 index traverse
            sentences[i] = re.sub(r'\W+', "", sentences[i])
        parsed_sentences = list()
        for sentence in sentences:
            words = list(jieba.cut(sentence, cut_all=False)) # 一句一句切
            parsed_sentences.append(words) # 切過的句子組合成文章
        # print(parsed_sentences)
        dataset.append(parsed_sentences) # 文章組合成檔案集 # parsed_sentences = article
    # print(dataset)
    return list(dataset)

In [3]:
def tf(article): # t: 詞
    word_statistics_in_article = collections.Counter()
    for sentence in article:
        word_statistics_in_article += collections.Counter(sentence)
    # print(word_statistics_in_article)
    ### 計算 tf
    total = sum(word_statistics_in_article.values(), 0.0)
    word_tf_in_article = {key: round((val/total), 2) for key, val in sorted(word_statistics_in_article.items(), key=lambda x: x[1], reverse=True)}
    # print(pd.DataFrame(list(word_tf_in_article.items()), columns=['word', 'tf'])) # DataFrame 好看但我不太會用
    return word_tf_in_article

In [4]:
def idf(tf_per_article_table, q): # terms: list of all words
    word_appear_count = collections.Counter()
    for i in range(len(tf_per_article_table)):
        word_appear_count += collections.Counter({key: 1 for key in tf_per_article_table[i].keys()})
    
    idf_table = {word: round(log10(q / count), 3) for word, count in word_appear_count.items()}
    return idf_table

In [120]:
a = {'A': 1, 'B': 1}
b = {'B': 1, 'C': 1}
# collections.Counter(a) + collections.Counter(b)
list(a.items())[:4]

[('A', 1), ('B', 1)]

In [5]:
def word_tf_in_dataset(dataset):
    all_words_count = collections.Counter()
    for article in dataset:
        for sentence in article:
            all_words_count += collections.Counter(sentence)
    total = sum(all_words_count.values(), 0.0)
    all_words_tf = {key: round((val/total), 3) for key, val in sorted(all_words_count.items(), key=lambda x: x[1], reverse=True)}
    return all_words_tf

In [6]:
def tf_idf(tf_t_d, idf_t_D):
    ### To Do
    # print(idf_t_D)
    table = dict()
    for t, tf in tf_t_d.items():
        # print(t, tf, idf_t_D[t])
        table[t] = round((tf * idf_t_D[t]), 3)
    # print(table)
    return {key: val for key, val in sorted(table.items(), key=lambda x: x[1], reverse=True)}

In [20]:
def rank_100(tf_idf_table):
    skip_flag = {key: False for key in tf_idf_table.keys()}
    rank_100 = {'a': 0.0}
    min_in_rank = ('a', 0.0)
    idx = 0
    while True:
        stop_flag = True
        for i in range(len(tf_idf_table)):
            if idx >= len(tf_idf_table[i].items()): # 第 i 篇文章已經掃完了, 直接往下一篇
                continue
            if skip_flag[i] != True: # 第 i 篇文章還沒掃完, 且不 skip
                word, weight = tuple(tf_idf_table[i].items())[idx] # list(tf_idf_table[i].items())[idx] 第 i 個文章中的 第 idx 大的
                if weight > min_in_rank[1]:  # if 文章 i 第一高權重的值 > rank 100 中最小值
                    rank_100[ word ] = weight # 加進新血
                    if len(rank_100) > 100: # 若列表超過 100, 該踢人 (ps: 還沒滿的話不排也沒差)
                        rank_100.pop(min_in_rank[0]) # 殺掉最小
                        new_rank_list = sorted(rank_100.items(), key=lambda x: x[1], reverse=True) # 重新排序後的列表
                        min_in_rank = new_rank_list[-1] # 更新最小
                        rank_100 = {key: val for key, val in new_rank_list} # 用重排後的 list 更新 rank_100
                    
                    stop_flag = False
                else:
                    skip_flag[i] = True
                
        if stop_flag == True: break # 都沒更新 === skip_flag 全部變 True 了, 跳出迴圈
        idx += 1
    print(rank_100, len(rank_100))
    return None

In [23]:
def main():
    file = open("./hw1-dataset.txt", mode='r')
    articles = file.readlines()
    dataset = preprocessing(articles)
    
    ### 統計高頻
    tf_whole_dataset_table = word_tf_in_dataset(dataset) # 高頻詞(已排序)
            
    ### 建 tf table
    tf_per_article_table = dict()
    for article_no in range(len(dataset)):
        tf_per_article_table[article_no] = tf(dataset[article_no])
    # print(tf_per_article_table) # 取值: tf_per_article_table[article_no][t] = tf(t, d)
    
    ### 建 idf table
    idf_table = idf(tf_per_article_table, len(dataset))
    # print(idf_table)
    
    ### 統計 tf-idf 高
    tf_idf_table = dict()
    for article_no in range(len(dataset)):
        tf_idf_table[article_no] = tf_idf(tf_per_article_table[article_no], idf_table)
    # print(tf_idf_table)
    rank_100(tf_idf_table)
    
    file.close()
main()

{'嘿嘿嘿': 2.885, '面議': 2.68, '廠廠': 2.642, '歐拉': 2.626, '霍霍': 2.28, '蛤': 2.212, '車手': 2.0, '之於': 2.0, '450000': 2.0, '啵': 2.0, 'Sowhatstheplan': 2.0, 'CARHAND': 2.0, 'Iwanttogonu': 2.0, '大嘴': 1.88, '金正恩': 1.849, '嫑': 1.699, '喔喔': 1.611, '曰': 1.56, '欸': 1.524, '一棟': 1.339, '憂傷': 1.32, '希拉蕊': 1.32, '顏楷': 1.32, 'edsheeran': 1.32, '鳳飛飛': 1.32, 'push': 1.32, 'MakeAmericaGreatAgain': 1.32, 'Thatheartiscold': 1.32, '林依晨': 1.221, '胖胖': 1.221, '蛋黃': 1.221, '準備好': 1.218, 'づ': 1.2, '姆咪': 1.198, '哈哈': 1.183, '孔雀': 1.163, '雷電': 1.16, 'Pavone': 1.16, 'agguy': 1.16, '色狼': 1.16, '吃吃': 1.121, '歐歐': 1.12, '皇民化': 1.08, '提琴': 1.08, '盡頭': 1.073, '恭喜': 1.063, '白冰冰': 1.063, '日': 1.061, '餓': 1.043, '新歌': 1.041, '黎明': 1.019, '22k': 1.005, 'Snowbaby': 1.0, '咕咕': 1.0, '888': 1.0, 'Ivebecomesonumb': 1.0, 'Letitbe': 1.0, '無情': 1.0, '隔宿': 1.0, '下注': 1.0, '帕': 1.0, '小熊': 1.0, 'KDKB': 1.0, '睡不著覺': 1.0, '蛤喇': 1.0, 'Ｔ': 1.0, '貌': 1.0, 'GetWild': 1.0, '壞運': 1.0, '親友團': 1.0, 'LaLaLaLaLaLoveSong': 1.0, '班馬班馬': 1.0, 'q': 1.0,

In [15]:
dataset = open("./hw1-dataset.txt", mode='r')
articles = list()
for article in dataset: # article: 一行算一個文章
    articles.append(article)
    
dataset.close()
print(articles[:3])

['為什麼 聖結石 會被酸而 這群人 不會？\t質感 劇本 成員 都差很多好嗎 不要拿腎結石來污辱這群人\n', '為什麼慶祝228會被罵可是慶端午不會？\t因為屈原不是台灣人，是楚國人。\n', '有沒有戰神阿瑞斯的八卦?\t爵士就是阿瑞斯 男主角最後死了\n']
